In [41]:
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sns
import itertools
import pyemma as pm
import matplotlib as mpl
import pandas as pd

In [42]:
sim = 'reactant'
data_dir  = f'final_bootstraps/{sim}'

In [43]:
def quantile(vals, q=0.5):
    tmp = np.quantile(vals, q=q, axis=0)
    return tmp

In [44]:
dt = 0.1
mod = pickle.load(open(data_dir+'/model_bs_0.p', 'rb'))
lag = (mod['model'].lag*dt)
lags = (np.arange(1, 51)/dt).astype(int)

In [45]:
mods = [pickle.load(x.open('rb')) for x in Path(data_dir).glob('*.p')]

In [46]:
cmods = [x['model'].pcca(2) for x in mods]

In [47]:
pops = np.concatenate([np.sort(x.coarse_grained_stationary_probability).reshape(1, -1) for x in cmods])

In [48]:
pops_m = quantile(pops)
pops_lb = quantile(pops, q=0.025)
pops_ub = quantile(pops, q=0.975)

In [49]:
results = pd.DataFrame({'mean': pops.mean(axis=0), 'median': pops_m, 'lb': pops_lb, 'ub': pops_ub})
results.index.name ='State'

In [50]:
(results*100).T.to_csv(f"{sim}_populations.csv", index=True)

In [51]:
t2s = [x['model'].timescales()[0] for x in mods]
t2s_m = quantile(t2s)*dt/1000
t2s_lb = quantile(t2s, q=0.025)*dt/1000
t2s_ub = quantile(t2s, q=0.975)*dt/1000

In [52]:
f"{t2s_m:4.2f} [{t2s_lb:4.2f},  {t2s_ub:4.2f}]"

'12.61 [1.04,  1615.38]'